In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv('../data/irtm.csv')

In [ ]:
df.head()

### Create dictionary

In [ ]:
from pa2 import make_dict
# dic = make_dict(df['content'])

In [ ]:
from pa2 import save_dict
# save_dict(dic, 'dictionary.txt')

### Create vectors

In [ ]:
from pa2 import create_vector
# create_vector(df['content'], list(df.index))

### Get dataset

In [ ]:
from pa3_NB import *

In [ ]:
fact = pd.factorize(df.object_class)

In [ ]:
len(fact[1])

In [ ]:
dataset = Dataset(np.arange(5739), fact[0])

In [ ]:
terms = read_dict(Config.dictionary_file)

In [ ]:
plt.hist([df for i, (string, df, cf) in terms], density=True)
plt.title('Document Frequency')
plt.show()

In [ ]:
plt.hist([cf for i, (string, df, cf) in terms], density=True)
plt.title('Collection Frequency')
plt.show()

### Feature Selection

In [ ]:
dictionary = max_seg_chi2(terms, dataset, size=150)

In [ ]:
clf = Classifier(dictionary)

In [ ]:
ma_train_f1, mi_train_f1, ma_valid_f1, mi_valid_f1 = cross_validation(clf, dataset, seed=1126)

In [ ]:
print(f'macro f1 (train): {ma_train_f1}')
print(f'micro f1 (train): {mi_train_f1}')
print(f'macro f1 (valid): {ma_valid_f1}')
print(f'micro f1 (valid): {mi_valid_f1}')

In [ ]:
clf.train(dataset)

In [ ]:
evaluate(clf, dataset)

### Confusion Matrix

In [ ]:
def confusion_matrix(clf, dataset, fold=10, permutation=True, seed=None):
  if seed:
    np.random.seed(seed)
    random.seed(seed)
  n = len(dataset)
  if permutation:
    permute = np.random.permutation(n)
  else:
    permute = np.arange(n)
  width = int(np.ceil(n / fold))
  left = 0
  right = width
  mat = np.zeros((Config.class_size, Config.class_size))
  for i in range(fold):
    train_permute = np.concatenate((permute[:left], permute[right:]))
    test_permute = permute[left:right]
#     test_permute = np.concatenate((permute[:left], permute[right:]))
#     train_permute = permute[left:right]
    left += width
    right += width
    dataset_train, dataset_test = train_test_split(dataset, train_permute, test_permute)
    clf.train(dataset_train)
    pred = clf.predict(dataset_test)
    for i in range(len(dataset_test)):
        mat[pred[i,1]-1, int(dataset_test.y[i]-1)] += 1
  mat /= fold
  return mat

In [ ]:
mat = confusion_matrix(clf, dataset, fold=10, seed=1126)

In [ ]:
import seaborn as sns
sns.heatmap(mat, annot=True,  linewidths=.5)
plt.show()

In [ ]:
print('Precision')
print(fact[1])
print(np.diag(mat) / np.sum(mat, axis=1))

In [ ]:
print('Recall')
print(fact[1])
print(np.diag(mat) / np.sum(mat, axis=0))

In [ ]:
print('F1')
print(fact[1])
p = np.diag(mat) / np.sum(mat, axis=1)
r = np.diag(mat) / np.sum(mat, axis=0)
print(2 * p * r / (p + r))

### Check the selected terms

In [ ]:
np.array(terms)[np.array(dictionary)-1]